### Analyse search terms on the e-commerce web server


In [3]:
# Install spark
!pip install pyspark
!pip install findspark
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 33.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.3-py2.py3-none-any.whl size=311885504 sha256=f988c84e9a3a18de12aada9a57f0ed98e6a149b3a906de43187b99f9327d290e
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/37/bc/bb/77785f6fcd2c83e663647f73225b76f3a3d5fd00762d7daf6f
Successfully built pyspark


In [4]:
import findspark
findspark.init()
import pandas as pd

In [5]:
from pyspark.sql           import SparkSession
from pyspark.ml.feature    import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark               import SparkContext, SparkConf

In [6]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("SparkML Ops").getOrCreate()

24/08/20 03:52:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv"

--2024-08-20 03:51:19--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2024-08-20 03:51:20 (43.1 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [7]:
# Load the csv into a spark dataframe
df       = pd.read_csv('searchterms.csv')
spark_df = spark.createDataFrame(df)

In [9]:
spark_df.show()

[Stage 0:>                                                          (0 + 1) / 1]

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|        laptop|
| 12|   11|2021|        laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| gaming laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|        laptop|
+---+-----+----+--------------+
only showing top 20 rows



In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg

In [10]:
baris = spark_df.count()
kolom = len(spark_df.columns)
print(f"Number of rows: ", baris)
print(f"Number of column: ", kolom)

[Stage 1:>                                                          (0 + 8) / 8]

Number of rows:  10000
Number of column:  4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [11]:
spark_df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [12]:
spark_df.select("searchterm").printSchema()

root
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [16]:
spark_df.createOrReplaceTempView("searchterm_laptop")

spark.sql("SELECT COUNT(*) as gaming_laptop FROM searchterm_laptop WHERE searchterm = 'gaming laptop'").show()

+-------------+
|gaming_laptop|
+-------------+
|          499|
+-------------+



In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [19]:
spark.sql(
    "SELECT searchterm, COUNT(*)as jumlah \
    FROM searchterm_laptop \
    GROUP BY searchterm \
    ORDER BY COUNT(*) \
    DESC LIMIT 5").show()

[Stage 12:===========================================>          (160 + 9) / 200]

+-------------+------+
|   searchterm|jumlah|
+-------------+------+
|mobile 6 inch|  2312|
|    mobile 5g|  2301|
|mobile latest|  1327|
|       laptop|   935|
|  tablet wifi|   896|
+-------------+------+



In [20]:
# The pretrained sales forecasting model is available at  the below url
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz"
!tar -xvzf model.tar.gz

--2024-08-20 04:03:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-08-20 04:03:52 (10.5 MB/s) - ‘model.tar.gz’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [21]:
model = LinearRegressionModel.load("sales_prediction.model")

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [22]:
# function to prediksi

def prediksi_penjualan_tahun(tahun):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[tahun, 0]]
    columns = ["year", "sales"]
    
    input_df = spark.createDataFrame(data, columns)
    assembled_df = assembler.transform(input_df).select("features", "year")
    
    prediksi = model.transform(assembled_df)
    prediksi.select("prediction").show()

In [23]:
prediksi_penjualan_tahun(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



24/08/20 04:10:04 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/08/20 04:10:04 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
